In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install sdv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8

In [3]:
# importing required libraries
import numpy as np
import pandas as pd

from sdv.single_table import GaussianCopulaSynthesizer
from sdv.metadata import SingleTableMetadata

from sdv.sampling import Condition

In [4]:
train_data = pd.read_csv('/content/drive/MyDrive/DLC-IDS/Datasets/train_merged_Probe.csv')
print(train_data.label.value_counts())

label
normal    67343
Other     46974
Probe     11656
Name: count, dtype: int64


In [5]:
attacks = ['Probe']

In [6]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(train_data)

# Step 1: Create the synthesizer
synthesizer = GaussianCopulaSynthesizer(metadata)

# Step 2: Train the synthesizer
synthesizer.fit(train_data)

/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [7]:
# SYNTHETIC DATA GENERATION WITH CONDITION
generated_df = pd.DataFrame(columns=train_data.columns)

for feature in attacks:
  synthetic_data_condition = Condition(
      num_rows= 30000,
      column_values={'label': feature}
  )

  synthetic_data = synthesizer.sample_from_conditions(
      conditions=[synthetic_data_condition]
  )

  print(synthetic_data.label.value_counts())
  generated_df = pd.concat([train_data, synthetic_data])
  print(generated_df.label.value_counts())

Sampling conditions: 100%|██████████| 30000/30000 [00:12<00:00, 2432.92it/s]

label
Probe    30000
Name: count, dtype: int64
label
normal    67343
Other     46974
Probe     41656
Name: count, dtype: int64


In [8]:
# saving the dataframe
generated_df.to_csv('/content/drive/MyDrive/DLC-IDS/Datasets/train_merged_Probe_sdv.csv', index = False)